### INTRODUTIONS
This is my first trial in **Jupyter** notebook. I had benifited from the covinience of Python coding by **Spyder** IDLE from Anaconda.  
This file is to analysis the performance persistancy of mutual funds and hedge funds if of which data are available.  
公募基金（私募基金）的业绩持续性研究。

*author: Qia.Ge  
Date: 2017-06-12*

### 0. Import Modules

In [2]:
# import modules
from WindPy import w # wind start
import numpy as np
import pandas as pd
import pymysql
import time
if w.isconnected()!=1:
    w.start()

Welcome to use Wind Quant API for Python (WindPy)!
You can use w.menu to help yourself to create commands(WSD,WSS,WST,WSI,WSQ,...)!

COPYRIGHT (C) 2016 WIND HONGHUI INFORMATION & TECHKNOLEWDGE CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


### 1. 数据选取
我们分析的基金范围是成立时间在2011年（5年以上）以前的股票型基金和偏股混合型基金  
样本基金从Wind编制普通股票型基金指数（885012.WI）和偏股混合型基金指数（885001.WI）中获取  
基金净值的时间范围为2011年至2016年，时间频度为年度

In [3]:
# basic settings
lasttradingDay ='2016-12-31'
fields  = "Period=Y;Fill=Previous"
minSetupDay = '2010-12-31'
# load fund code list
def get_fund_list(fundIndexname):
    wsetdata = w.wset("sectorconstituent",'date='+lasttradingDay, 'windcode='+fundIndexname)
    codelist = wsetdata.Data[1]
    fundsetupday = w.wss(codelist,'fund_setupdate')
    fundsetupdaylist = [x.strftime('%Y-%m-%d') for x in fundsetupday.Data[0]]
    return list(np.array(codelist)[np.where([x < minSetupDay for x in fundsetupdaylist])]) # setup date ahead of min setup day
# load fund nav
def get_fund_nav(fundCodelist):
    wsddata = w.wsd(fundCodelist,"NAV_adj",minSetupDay,lasttradingDay,fields)
    return pd.DataFrame(wsddata.Data,index=wsddata.Codes,columns=wsddata.Times).T

### 2.股票型基金
#### （1）获取普通股票型基金的代码、净值，并以此计算其复权单位净值增长率

In [245]:
# table 0
# 原始数据
stockFundCodelist = get_fund_list('885012.WI')
stockFundNav = get_fund_nav(stockFundCodelist)
stockFundRet = ((stockFundNav/stockFundNav.shift(1)-1).dropna()).iloc[-5:] # we drop fund return data of year 2011 
stockFundRet

,000051.OF,020011.OF,040002.OF,040180.OF,040190.OF,050002.OF,050013.OF,090010.OF,090012.OF,100032.OF,...,519116.OF,519180.OF,519300.OF,519671.OF,519686.OF,530010.OF,540006.OF,540007.OF,540008.OF,540009.OF
2012-12-31 00:00:00.005,0.094675,0.079167,0.083396,0.114667,0.143038,0.082508,0.163793,0.071970,0.006702,0.100600,...,0.086782,0.111877,0.084790,0.134393,0.135766,0.142052,0.049834,0.014418,-0.006245,0.039580
2013-12-31 00:00:00.005,-0.050000,-0.058108,-0.040619,-0.065789,-0.100775,-0.063063,-0.202469,-0.075383,0.025300,-0.069194,...,-0.077396,-0.072881,-0.055136,-0.091720,-0.075835,-0.076012,0.103622,0.155802,0.414157,0.085781
2014-12-31 00:00:00.005,0.506401,0.460545,0.489919,0.588604,0.407635,0.577686,0.487616,0.515924,0.016883,0.484725,...,0.479361,0.587809,0.495126,0.643759,0.650904,0.605876,0.602522,0.380868,0.095246,-0.055411
2015-12-31 00:00:00.005,0.088763,0.072692,0.100710,0.036753,0.190726,0.186998,-0.010175,0.252941,0.458493,0.304199,...,0.116112,-0.002926,0.079870,0.037543,-0.024431,0.023618,0.351822,0.080408,0.583480,0.167383
2016-12-30 00:00:00.005,-0.079792,-0.078231,-0.101637,-0.074322,-0.099927,-0.037024,0.019390,-0.044936,-0.180385,-0.005859,...,-0.046774,-0.076770,-0.095111,-0.024671,-0.056995,-0.049201,0.064600,-0.110565,-0.239207,-0.063330


#### （2）梯队编制
将所有基金分别按照每年基金的复权单位净值增长率分为5组，标记为第1~5梯队；当年收益最高的编入第1梯队，按收益递减依次编入下一梯队，最差组合编入第5梯队。统计每组的逐年平均收益率和收益率标准差，并保存

In [246]:
namelist = ['第5梯队','第4梯队','第3梯队','第2梯队','第1梯队']

# 定义分组平均收益率统计函数
def my_group(x):
    grouping  = pd.qcut(x,5,labels=namelist) # 首先将对象按照分位数进行切分
    return x.groupby(grouping).mean() # 然后统计分组平均收益率

# table 1
# 计算每组的逐年平均收益率
stockFundRetrank = stockFundRet.apply(my_group,axis=1) 
stockFundRetrank.index = stockFundRetrank.index.strftime('%Y')
stockFundRetrank = stockFundRetrank.T
print(stockFundRetrank)
#stockFundRetrank.to_csv('E:/stockFundRetrank.csv')

# table 2
# 定义分组平均收益率标准差统计函数
my_group_qcut_std = lambda x: x.groupby(pd.qcut(x,5,labels=namelist)).std()
stockFundRetstd = stockFundRet.apply(my_group_qcut_std,axis=1)
stockFundRetstd.index = stockFundRetstd.index.strftime('%Y')
stockFundRetstd = stockFundRetstd.T
print(stockFundRetstd)
#stockFundRetstd.T.to_csv('e:/stockFundRetstd.csv')

                                2012      2013      2014      2015      2016
2016-12-30 00:00:00.005000                                                  
第1梯队                        0.155997  0.168772  0.673051  0.391195 -0.001592
第2梯队                        0.108919 -0.026183  0.576581  0.189527 -0.049657
第3梯队                        0.081675 -0.066462  0.506893  0.087951 -0.071987
第4梯队                        0.050680 -0.089214  0.392956  0.035221 -0.098093
第5梯队                        0.011593 -0.147699  0.184293 -0.024743 -0.168330
                                2012      2013      2014      2015      2016
2016-12-30 00:00:00.005000                                                  
第1梯队                        0.027266  0.082050  0.060424  0.116117  0.031411
第2梯队                        0.010088  0.032608  0.019303  0.033616  0.007852
第3梯队                        0.005202  0.007167  0.009524  0.017949  0.005932
第4梯队                        0.008262  0.007690  0.057774  0.016736  0.013606

#### （3）转移概率矩阵
**a.** 统计每个梯队在本年及以后年度的排名情况

In [247]:
# 首先应得到每只基金在每个年度的排名梯队情况
stockFundRet.apply(np.argsort,axis=1) # 这个axis有点奇怪，如果写成0就是每只基金自己按年份排序，奇怪
# 但是这个矩阵无法反应每只基金所在的分组，应当改写下
def my_fund_rank(x):
    # x : series
    x0 = [x.quantile(.2),x.quantile(.4),x.quantile(.6),x.quantile(.8)]
    y = np.ones_like(x)
    y[np.where(x<=x0[1])and np.where(x>x0[0])] = 2
    y[np.where(x<=x0[2])and np.where(x>x0[1])] = 3
    y[np.where(x<=x0[3])and np.where(x>x0[2])] = 4
    y[np.where(x>x0[3])] = 5
    return y
# 每年的截面排名
# 注：这个里面数值越大越好，相当于得分
stockFundRankMatrx = stockFundRet.apply(my_fund_rank,axis=1)
stockFundRankMatrx.index = stockFundRankMatrx.index.strftime('%Y')
stockFundRankMatrx

,000051.OF,020011.OF,040002.OF,040180.OF,040190.OF,050002.OF,050013.OF,090010.OF,090012.OF,100032.OF,...,519116.OF,519180.OF,519300.OF,519671.OF,519686.OF,530010.OF,540006.OF,540007.OF,540008.OF,540009.OF
2012,4.0,3.0,3.0,4.0,5.0,3.0,5.0,3.0,1.0,4.0,...,3.0,4.0,3.0,5.0,5.0,5.0,2.0,1.0,1.0,2.0
2013,4.0,3.0,4.0,3.0,2.0,3.0,1.0,3.0,4.0,3.0,...,2.0,3.0,4.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0
2014,3.0,2.0,3.0,4.0,2.0,4.0,2.0,3.0,1.0,2.0,...,2.0,4.0,3.0,5.0,5.0,4.0,4.0,2.0,1.0,1.0
2015,3.0,3.0,3.0,2.0,4.0,4.0,1.0,5.0,5.0,5.0,...,3.0,1.0,3.0,2.0,1.0,2.0,5.0,3.0,5.0,4.0
2016,3.0,3.0,2.0,3.0,2.0,4.0,5.0,4.0,1.0,5.0,...,4.0,3.0,2.0,5.0,4.0,4.0,5.0,2.0,1.0,3.0


**b.** 根据条件概率公式P(B|A) = P(AB)/P(A)计算每年度的转移概率

In [9]:
# 尝试写下计算的核心代码
def my_cond_prob_cal(x0,x1):
    # x0 : last year rank
    # x1: present year rank
    rankset = [1,2,3,4,5]
    rankres = []
    # 这个排名里面，i越小说明收益率越低
    for i in rankset:
        Pa = len(set(list(np.where(x0==i)[0])))
        for j in rankset:
            Pab = len(set(list(np.where(x0==i)[0])) & set(list(np.where(x1==j)[0])))
            rankres.append(Pab/Pa)
    return rankres
# 改写上述代码，希望生成的是DataFrame
def my_genr_name_chg():
    # 生成档位分组和档位变化列表，后者长度为 5*5
    chgres  = []
    nameres = []
    rankset = [1,2,3,4,5]
    for i in rankset:
        for j in rankset:
            nameres.append('第'+str(5+1-i)+'梯队')
            if j == i:
                chgres.append('无变化')
            elif j > i: # 排名上升n=j-i个梯队
                chgres.append('前进'+str(j-i)+'档')
            elif j < i:
                chgres.append('后退'+str(i-j)+'档')
    return nameres,chgres
def my_trans_prob_cal(fDataFrame):
    # fDataFrame:fund rank matrix of each year, e.g stockFundRankMatrx
    transProbMatrix = pd.DataFrame()
    transProbMatrix['Name'],transProbMatrix['Chg'] = my_genr_name_chg()
    for t in (np.arange(len(fDataFrame.index))+1)[:-1]:
        transProbMatrix['Year'+str(fDataFrame.index[t])] = my_cond_prob_cal(fDataFrame.iloc[t-1,:],fDataFrame.iloc[t,:])
    ## 可用下面代码验证概率计算是否有误
    ## 结果中的数据应该全为1
    #transProbMatrix.groupby('Name').sum(axis=0)
    return transProbMatrix


In [ ]:
# table 3
stockfundtransProbMatrix = my_trans_prob_cal(stockFundRankMatrx)
stockfundtransProbMatrix
#stockfundtransProbMatrix.to_csv('e:/stockfundtransProbMatrix.csv')

**c.** 汇总每年度的转移概率，得到平均后的不同梯队间的转移概率矩阵

In [249]:
# table 4
# 公募基金不同梯队的概率转移矩阵
stockAvgTransProbMatrix = pd.DataFrame(stockfundtransProbMatrix.mean(axis=1).values.reshape(5,5),index=namelist,columns=namelist)
# 顺序是按照第5梯队到第1梯队倒序排列的，现转换为正常顺序
stockAvgTransProbMatrix = (stockAvgTransProbMatrix[stockAvgTransProbMatrix.columns.sort_values()].T)[stockAvgTransProbMatrix.columns.sort_values()]
stockAvgTransProbMatrix

,第1梯队,第2梯队,第3梯队,第4梯队,第5梯队
第1梯队,0.052632,0.039474,0.092105,0.381579,0.4125
第2梯队,0.052632,0.171053,0.144737,0.263158,0.3500
第3梯队,0.052632,0.236842,0.473684,0.131579,0.1000
第4梯队,0.355263,0.144737,0.236842,0.157895,0.1000
第5梯队,0.486842,0.407895,0.052632,0.065789,0.0375


### 3. 偏股混合型基金
#### (1) 获取基金代码、净值，计算复权单位净值收益率

In [253]:
# table 3-0
# 原始数据
preferStockFundNav = get_fund_nav(get_fund_list('885001.WI'))
preferStockFret = (preferStockFundNav/preferStockFundNav.shift(1)-1).dropna()[-5:]
preferStockFret

,000001.OF,000011.OF,000021.OF,000031.OF,000061.OF,002011.OF,020001.OF,020003.OF,020005.OF,020009.OF,...,630001.OF,630002.OF,630006.OF,660001.OF,660003.OF,660004.OF,660005.OF,660006.OF,690003.OF,690004.OF
2012-12-31 00:00:00.005,0.072545,0.058866,0.008264,-0.006598,0.112903,0.019780,0.041940,-0.121791,0.024425,0.031980,...,0.172803,-0.045064,0.016641,0.141226,0.024930,0.113995,0.111273,0.068343,0.041607,0.052308
2013-12-31 00:00:00.005,0.157128,0.149198,0.132969,0.309298,-0.018445,0.129310,0.218868,0.124138,-0.025245,0.089392,...,0.375291,0.290637,0.174107,0.416305,0.205808,0.180978,0.437418,0.137839,-0.022039,0.103801
2014-12-31 00:00:00.005,0.152688,0.060710,0.112540,0.165942,0.238926,0.225827,0.233649,0.085890,0.113669,0.438731,...,0.307947,0.220546,0.247148,0.130625,0.155194,0.095044,0.098322,0.012199,0.171831,0.312583
2015-12-31 00:00:00.005,0.258621,0.281671,0.431358,0.328154,0.318527,0.524650,0.527309,0.814686,0.087855,0.186170,...,0.292763,0.846675,0.584350,0.501437,0.493024,0.413950,0.821998,0.165689,0.608173,0.535822
2016-12-30 00:00:00.005,-0.227165,-0.112870,-0.202928,-0.144595,-0.143796,-0.196052,-0.009126,0.033445,-0.005938,-0.048545,...,-0.050993,-0.331264,-0.120959,-0.203306,-0.114594,-0.132877,-0.201529,-0.140879,-0.187593,-0.140604


#### (2) 分组编制梯队

In [255]:
# table 3-1 & 3-2
# 计算每组的逐年平均收益率和标准差
my_group_qcut_mean = lambda x: x.groupby(pd.qcut(x,5,labels=namelist)).mean()
my_group_qcut_std = lambda x: x.groupby(pd.qcut(x,5,labels=namelist)).std()
pSRetRank = preferStockFret.apply(my_group_qcut_mean,axis=1)
pSRetRank.index = preferStockFret.index.strftime('%Y')
pSRetRank = pSRetRank.T
print(pSRetRank)
pSRetstd = preferStockFret.apply(my_group_qcut_std,axis=1)
pSRetstd.index = preferStockFret.index.strftime('%Y')
pSRetstd = pSRetstd.T
print(pSRetstd)

                                2012      2013      2014      2015      2016
2016-12-30 00:00:00.005000                                                  
第1梯队                        0.147890  0.343123  0.457170  0.741904 -0.001814
第2梯队                        0.085986  0.194090  0.295405  0.558731 -0.094850
第3梯队                        0.048038  0.133534  0.212029  0.434764 -0.147155
第4梯队                        0.018830  0.079177  0.140546  0.314470 -0.200700
第5梯队                       -0.035067 -0.023326  0.047613  0.165982 -0.283700
                                2012      2013      2014      2015      2016
2016-12-30 00:00:00.005000                                                  
第1梯队                        0.039209  0.100992  0.089375  0.104712  0.052365
第2梯队                        0.013698  0.022298  0.023568  0.040203  0.013365
第3梯队                        0.008455  0.013583  0.024414  0.034872  0.015798
第4梯队                        0.009543  0.016370  0.021817  0.036196  0.013688

#### (3) 转移概率矩阵

In [263]:
pSRetScoreMatrix = preferStockFret.apply(my_fund_rank,axis=1)
pSRetScoreMatrix.index = pSRetScoreMatrix.index.strftime('%Y')
# table 3-3
pSTransProbMatrix = my_trans_prob_cal(pSRetScoreMatrix)
print(pSTransProbMatrix)
# table 3-4
# 公募基金不同梯队的概率转移矩阵
pSAvgTransProbM = pd.DataFrame(pSTransProbMatrix.mean(axis=1).values.reshape(5,5),index=namelist,columns=namelist)
pSAvgTransProbM = (pSAvgTransProbM[pSAvgTransProbM.columns.sort_values()].T)[pSAvgTransProbM.columns.sort_values()]
print(pSAvgTransProbM)

    Name   Chg  Year2013  Year2014  Year2015  Year2016
0   第5梯队   无变化  0.200000  0.183333  0.250000  0.266667
1   第5梯队  前进1档  0.183333  0.216667  0.150000  0.183333
2   第5梯队  前进2档  0.233333  0.133333  0.183333  0.283333
3   第5梯队  前进3档  0.250000  0.166667  0.166667  0.100000
4   第5梯队  前进4档  0.133333  0.300000  0.250000  0.166667
5   第4梯队  后退1档  0.133333  0.216667  0.150000  0.183333
6   第4梯队   无变化  0.200000  0.166667  0.216667  0.200000
7   第4梯队  前进1档  0.216667  0.166667  0.233333  0.216667
8   第4梯队  前进2档  0.183333  0.266667  0.233333  0.216667
9   第4梯队  前进3档  0.266667  0.183333  0.166667  0.183333
10  第3梯队  后退2档  0.183333  0.150000  0.100000  0.216667
11  第3梯队  后退1档  0.200000  0.183333  0.183333  0.183333
12  第3梯队   无变化  0.200000  0.233333  0.216667  0.150000
13  第3梯队  前进1档  0.250000  0.250000  0.183333  0.250000
14  第3梯队  前进2档  0.166667  0.183333  0.316667  0.200000
15  第2梯队  后退3档  0.266667  0.200000  0.166667  0.200000
16  第2梯队  后退2档  0.266667  0.150000  0.250000  0.250000
17  第2梯队  

### 4. 私募基金
从朝阳永续私募基金数据库提取股票多头/股票多空类别下的私募基金。时间维度仍与此前保持一致。
#### （1） 数据提取
**朝阳永续私募基金数据库**  
需要提取基金的代码、简称、复权单位净值、成立日期、净值披露频率、基金运行状态  
选取**成立时间早于2016年1月1日**且处于**存续状态**的基金，数据披露频率为**周度**，投资类型为**股票多头**  
注：模拟账号只能提取最近2年的私募基金数据。

In [79]:
# 数据查询比较耗时，引入计时模块
t0 = time.clock()
# database settings
config = {
          'host':'106.75.45.237',
          'port':15628,
          'user':'simu_dycy',
          'password':'A16IJemwZlqw9Pl6',
          'db':'CUS_FUND_DB',
          'charset':'utf8',
          'cursorclass':pymysql.cursors.DictCursor,
          }
conn = pymysql.connect(**config)
cur = conn.cursor()
# select data 
sqlquery = """
SELECT a.fund_id as 'Code',
a.fund_name as 'Name',
a.data_freq as '净值披露频率',
a.fund_type_strategy as '投资策略',
b.swanav as 'Nav',
b.statistic_date as '统计日期'
FROM v2_fund_info a,
v2_fund_nv_data_zyyx b
WHERE a.fund_id = b.fund_id
AND a.fund_status = '存续中'
AND a.data_freq  = '周'
AND a.fund_type_strategy ='股票多头'
AND a.foundation_date < '2016-01-01'
AND b.statistic_date between '2016-01-01' and '2017-05-31'
"""
cur.execute(sqlquery)
results = cur.fetchall()
res_info = pd.DataFrame(results)
cur.close()
conn.close()
print('%.2fseconds process time'%(time.clock()-t0))
#res_info
#res_info.to_csv('e:/朝阳永续私募基金数据.csv')

77.24seconds process time


,Code,Name,Nav,净值披露频率,投资策略,统计日期
0,36568,华融金三板掘金1号,0.946600,周,股票多头,2016-01-08
1,36568,华融金三板掘金1号,0.905200,周,股票多头,2016-01-15
2,36568,华融金三板掘金1号,0.896000,周,股票多头,2016-01-22
3,36568,华融金三板掘金1号,0.859700,周,股票多头,2016-01-29
4,36568,华融金三板掘金1号,0.867600,周,股票多头,2016-02-05
5,36568,华融金三板掘金1号,0.858900,周,股票多头,2016-02-19
6,36568,华融金三板掘金1号,0.838900,周,股票多头,2016-02-26
7,36568,华融金三板掘金1号,0.817900,周,股票多头,2016-03-04
8,36568,华融金三板掘金1号,0.807400,周,股票多头,2016-03-11
9,36568,华融金三板掘金1号,0.813800,周,股票多头,2016-03-18


#### （2）数据处理
需要将下载的数据格式和顺序做一些转换：DataFrame中纵轴为统计日期，横轴为基金代码；转换的思想为生成新的dataframe然后Merge。  
DataFrame数据中选择周五作为基金净值的制定披露日期.  
净值的计算以月为频度，由于转移概率矩阵频度暂时设定为季度，因此仍需要进行**频度转换**  

In [261]:
# generate date list as index of Hedge Fund DataFrame
# we set Friday as the date that hedge fund diclosure nav
HFdatelist = pd.date_range(start='2016-01-01',end='2017-05-31',freq='W-FRI') 
# generate hedge fund code list
HFcodelist = res_info['Code'].drop_duplicates()
HFcodelist[:2]
HFdata = pd.DataFrame(index = HFdatelist)
for codei in HFcodelist[:100]:
    fidata = pd.DataFrame(res_info['Nav'][res_info['Code']==codei])
    fidata.index = res_info['统计日期'][res_info['Code']==codei]
    fidata.columns=[codei]
    HFdata = (HFdata.join(fidata.dropna(),how='left')).fillna(method='ffill')
HFret = pd.DataFrame((HFdata.resample('Q').last()).pct_change().dropna(axis=0,how='all'))

#### （3）梯队分组

In [ ]:
# table 4-1 & 4-2
# 计算每组的逐年平均收益率和标准差
namelist = ['第5梯队','第4梯队','第3梯队','第2梯队','第1梯队']
my_group_qcut_mean = lambda x: x.groupby(pd.qcut(x,5,labels=namelist)).mean()
my_group_qcut_std = lambda x: x.groupby(pd.qcut(x,5,labels=namelist)).std()
HFretRank = HFret.apply(my_group_qcut_mean,axis=1)
HFretRank.index = HFret.index.strftime('%Y')
HFretRank = HFretRank.T
print(HFretRank)
HFretstd = HFret.apply(my_group_qcut_std,axis=1)
HFretstd.index = HFret.index.strftime('%Y')
HFretstd= HFretstd.T
print(HFretstd)

#### decimal 数据会报错，无解！！！！

### 5. 不同时间频度下的业绩持续性研究
分别研究基金的1年、3年、5年业绩持续性问题
考虑不同观察期和持有期组合下基金业绩的持续性，如12M-12M、36M-9M等.  
如果观察期为3年，则基金成立时间应早于2008-12-31

In [15]:
def stock_ret_persist(N):
    minSetupDay = '2010-12-31'
    namelist = ['第5梯队','第4梯队','第3梯队','第2梯队','第1梯队']

    # 原始数据
    stoFundCodelist = get_fund_list('885012.WI')
    stoFundNav = get_fund_nav(stoFundCodelist)
    stoFundRet = ((stoFundNav/stoFundNav.shift(N)-1).dropna()).iloc[-5:] # 3-year ret

    # table 5-1 & 5-2
    # 计算每组的逐年平均收益率和标准差
    my_group_qcut_mean = lambda x: x.groupby(pd.qcut(x,5,labels=namelist)).mean()
    my_group_qcut_std = lambda x: x.groupby(pd.qcut(x,5,labels=namelist)).std()

    stoFundRetRank = stoFundRet.apply(my_group_qcut_mean,axis=1)
    stoFundRetRank.index = stoFundRet.index.strftime('%Y')
    stoFundRetRank = stoFundRetRank.T
    print(stoFundRetRank)
    stoFundRetstd = stoFundRet.apply(my_group_qcut_std,axis=1)
    stoFundRetstd.index = stoFundRet.index.strftime('%Y')
    stoFundRetstd = stoFundRetstd.T
    print(stoFundRetstd)

    def my_fund_rank(x):
        # x : series
        x0 = [x.quantile(.2),x.quantile(.4),x.quantile(.6),x.quantile(.8)]
        y = np.ones_like(x)
        y[np.where(x<=x0[1])and np.where(x>x0[0])] = 2
        y[np.where(x<=x0[2])and np.where(x>x0[1])] = 3
        y[np.where(x<=x0[3])and np.where(x>x0[2])] = 4
        y[np.where(x>x0[3])] = 5
        return y

    stoFundScoreMatrx = stoFundRet.apply(my_fund_rank,axis=1)
    stoFundScoreMatrx.index = stoFundScoreMatrx.index.strftime('%Y')
    # table 5-3
    stoFundTransProbMatrix = my_trans_prob_cal(stoFundScoreMatrx)
    print(stoFundTransProbMatrix)
    # table 5-4
    # 公募基金不同梯队的概率转移矩阵
    stoAvgTransProbM = pd.DataFrame(stoFundTransProbMatrix.mean(axis=1).values.reshape(5,5),index=namelist,columns=namelist)
    stoAvgTransProbM = (stoAvgTransProbM[stoAvgTransProbM.columns.sort_values()].T)[stoAvgTransProbM.columns.sort_values()]
    print(stoAvgTransProbM)


In [19]:
stock_ret_persist(3)
#stock_ret_persist(5)

          2013      2014      2015      2016
第1梯队 -0.051592  0.730215  1.000047  0.800095
第2梯队 -0.161499  0.602733  0.646867  0.564209
第3梯队 -0.204791  0.548900  0.533561  0.495925
第4梯队 -0.229719  0.490646  0.479171  0.442414
第5梯队 -0.317812  0.228858  0.295663  0.270543
          2013      2014      2015      2016
第1梯队  0.070132  0.056598  0.208161  0.166201
第2梯队  0.016196  0.020025  0.069210  0.045616
第3梯队  0.010307  0.012398  0.011655  0.012668
第4梯队  0.008432  0.034472  0.024878  0.024484
第5梯队  0.082341  0.159307  0.170762  0.087468
    Name   Chg  Year2014  Year2015  Year2016
0   第5梯队   无变化  0.650000  0.450000  0.350000
1   第5梯队  前进1档  0.100000  0.100000  0.150000
2   第5梯队  前进2档  0.100000  0.200000  0.400000
3   第5梯队  前进3档  0.100000  0.150000  0.100000
4   第5梯队  前进4档  0.050000  0.100000  0.000000
5   第4梯队  后退1档  0.157895  0.157895  0.105263
6   第4梯队   无变化  0.421053  0.315789  0.421053
7   第4梯队  前进1档  0.157895  0.105263  0.210526
8   第4梯队  前进2档  0.210526  0.157895  0.210526
9   第4梯队  

### 6. 其他研究业绩相关性的方法
Spearman秩相关检验、

### 7. 基金经理的业绩持续性研究
以上研究均为基于具体的基金产品。但实际上，金融产品收益能力的背后是基金经理的投资能力。对基金经理的投资能力进行分析是公募基金投资能力评价中的重要部分。我们选择投资年限在5年以上的基金经理作为分析对象。将其近5年掌管的所有公募基金的收益，按投资类别等权重汇总到其个人名下，制作其个人投资能力净值曲线，并以此为基础，对基金经理的投资业绩持续性进行评价。

#### 7.1 

### 8.未来研究方向
能否以剔除了风格影响后的alpha作为基金投资能力的指标而不是简单的以复权单位净值收益率？